In [73]:
import os
import requests
import requests
import ast

import pandas as pd
import glob
import plotly.express as px

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from time import sleep


from locations import *
from parsing import *

# LOCATIONS SCRAPING

In [74]:
# Base

base_url = 'https://www.pisos.com/'

venta_url = f'{base_url}venta/'

In [75]:
# Sacamos todas las paginas filtradas por provincias

response = requests.get(base_url)

soup = BeautifulSoup(response.text, "html.parser")

provincias_str = soup.find('div', class_ = 'home-container').find('div', class_ = 'selectBox').find_all('ul')[-1].text

provincias = [x.lower() for x in provincias_str.strip().split('\n')]

endpoints = replaceWithUnderscore(provincias)

endpoints = [venta_url + endpoint for endpoint in endpoints]

with open('locations.txt', 'w') as file:
    file.write('\n'.join(endpoints))

# SINGLE PAGE SCRAPING

In [76]:
def scrape(url):

    browser = webdriver.Chrome()
    browser.get(url)
    browser.maximize_window()

    element = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="didomi-notice-agree-button"]'))
    )

    element.click() # Accept cookies

    for i in range(30):

        element = WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.XPATH, f'(//*[@id]/div[2]/div[1]/div[2]/a)[{i + 1}]'))
        )

        element.click() # Click on the first post

        # <SCROLLING>
        while True:
            is_at_bottom = browser.execute_script("return window.scrollY + window.innerHeight >= document.body.scrollHeight")
            browser.execute_script("window.scroll({ top: document.body.scrollHeight, behavior: 'smooth' });")
            if is_at_bottom:
                break
            sleep(0.2)
        # </SCROLLING>
        
        html_content = browser.page_source
        soft_url = url.replace('https://www.pisos.com/venta/', '')
        soft_url = soft_url.replace('/', '_')
        soft_url = soft_url.replace('-', '_')

        file_path = f'{os.getcwd()}/html_content/{soft_url}_{i}.html'
        try:
            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(html_content)
        except Exception as e:
            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(repr(e))

        go_back_element = WebDriverWait(browser, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="GoBackLink"]'))
        )

        go_back_element.click() # Click on back arrow

    browser.quit()

    

# TESTING

In [77]:
def scanRegions(url):
    response = requests.get(url)

    print(f'URL: {url} | STATUS {response.status_code}')

    soup = BeautifulSoup(response.text, 'html.parser')

    items = soup.select('div.zoneList a.item:not(.item-subitem)')

    endpoints = {}
    for item in items:
        endpoint = item['href']
        n_results = item.find('span', class_ = 'total').text

        # <INT>
        if len(n_results) != 0:
            try:
                n_results = n_results[1:-1]
                n_results = ''.join(n_results.split('.'))
                n_results = int(n_results)
            except:
                print(f'FAIL CASTING TO INTEGER {endpoint}')
        else:
            print(f'n_results EMPTY {endpoint}')
        #</INT>

        # <RECURSSION>
        if n_results > 3000:
            endpoints[endpoint] = scanRegions(base_url[:-1] + endpoint)
        # </RECURSSION>

        else: endpoints[endpoint] = n_results

    return endpoints

In [78]:
def parseRegions(endpoints):

    def extract(endpoints):
        array = []
        for key, value in endpoints.items():
            if isinstance(value, int):
                array.append(key)
                continue

            data = extract(value)
            array.extend(data)

        return array
    
    endpoints = extract(endpoints)

    array = []
    
    for endpoint in endpoints:
        data = endpoint
        if '/venta/pisos-' in endpoint:
            data = endpoint.replace('/venta/pisos-', '/viviendas/')
        array.append(data)

    return array

In [79]:
# URL = 'https://www.pisos.com/viviendas/o_eume_ortegal/'

# urls = scanRegions(URL)
# urls = parseRegions(urls)
# urls = [base_url + x for x in urls]
# urls = [x.replace('//viviendas/', '/venta/pisos-') for x in urls]
# print(urls)

In [80]:
# for url in urls:
#     for i in range(20):
#         try:
#             endpoint = url + f'{i+1}'
#             scrape(endpoint)
#         except:
#             break

In [81]:
# data = {
#     'price' : [],
#     'title' : [],
#     'location' : [],
#     'lat' : [],
#     'lng' : [],
#     'characteristics' : [],
# }

# files = glob.glob('html_content/*.html')

# for file in files:
#     with open(file, encoding='utf-8') as f:
#         source = f.read()
#     soup = BeautifulSoup(source, 'html.parser')

#     price = getPrice(soup)
#     title = getTitle(soup)
#     location = getLocation(soup)
#     lat, long = getLatLong(soup)
#     characteristics = getCharacteristics(soup)

#     data['price'].append(price)
#     data['title'].append(title)
#     data['location'].append(location)
#     data['lat'].append(lat)
#     data['lng'].append(long)
#     data['characteristics'].append(characteristics)

# df = pd.DataFrame(data)

In [82]:
df = pd.read_csv('prueba.csv')

In [83]:
def tryInt(n):
    try:
        return int(n)
    except: return np.nan

# DATA PROCESSING

In [84]:
df['price'] = df['price'].map(lambda x: tryInt(''.join(x[:-2].split('.'))))
df['characteristics'] = df['characteristics'].apply(ast.literal_eval)

#### CHARACTERISTICS PARSING

In [85]:
# HAY QUE REFACTORIZAR ESTE CHURRO HORRIBLE!!!!!!!

columns = []
characteristics = df['characteristics'].iloc

for chars in characteristics:
    for char in chars:
        vals = char.split(':')
        if isinstance(vals, list):
            columns.append(vals[0])
        else:
            columns.append(vals)

columns = [x.strip() for x in columns]
columns = list(set(columns))
data = []

for chars in characteristics:

    dict_data = {}

    for char in chars:
        content = char.split(':')
        if len(content) == 2:
            key, value = content
            key = key.strip()
            dict_data[key] = value
        else:
            continue

    dict_columns = {}

    for column in columns:
        for key, value in dict_data.items():
            if key == column:
                dict_columns[column] = value
        
    for column in columns:
        if dict_columns.get(column) is None:
            dict_columns[column] = np.nan

    data.append(dict_columns)

# HAY QUE REFACTORIZAR ESTE CHURRO HORRIBLE!!!!!!!

In [86]:
df_characteristics = pd.DataFrame(data)

In [87]:
df_ = pd.concat([df, df_characteristics], axis = 1).drop('characteristics', axis = 1)

In [88]:
df_

,price,title,location,lat,lng,Superficie construida,Baños,Orientación,Superficie solar,Superficie útil,...,Amueblado,Sistema de seguridad,Carpintería interior,Puerta blindada,Calle asfaltada,Gastos de comunidad,Tipo de casa,Carpintería exterior,Gas,Portero automático
0,180000.0,Finca rústica en venta en Calle Lugar Babio,Calle Lugar Babio. Bergondo (Carrio) (Bergondo),43.317188,-8.243459,445 m²,1,Norte,2.921 m²,400 m²,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,210000.0,Casa en venta en Calle Punta Do Salto,Calle Punta Do Salto. Arteixo,43.340884,-8.482044,420 m²,3,NaN,626 m²,360 m²,...,Semi amueblado,NaN,NaN,NaN,NaN,NaN,NaN,Aluminio,NaN,NaN
2,126626.0,Piso en venta en Calle Blanco White,Calle Blanco White. Os Mallos-A Falperra (A Co...,43.347553,-8.416538,78 m²,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,102600.0,Piso en venta en Calle del Roi Xordo,Calle del Roi Xordo. Culleredo,43.318661,-8.384513,104 m²,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,95000.0,Piso en venta en Travesía de Pastoriza,Travesía de Pastoriza. Arteixo,43.336506,-8.462301,93 m²,2,Norte,NaN,83 m²,...,NaN,NaN,NaN,NaN,NaN,Entre 40 y 60€,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4214,479900.0,Casa en venta en Calle Casteliño,Calle Casteliño. Sada,43.339523,-8.267679,465 m²,4,NaN,2.715 m²,392 m²,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4215,145000.0,Piso en venta en Calle de la obra,Calle de la obra. Sada,43.348638,-8.251601,102 m²,2,Sur,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Entre 40 y 60€,NaN,NaN,NaN,NaN
4216,134000.0,Casa unifamiliar en venta en Avenida de Sada y...,"Avenida de Sada y sus Contornos, 44, cerca de ...",43.358830,-8.251664,87 m²,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Entre 10 y 20 €,Unifamiliar,NaN,NaN,NaN
4217,380000.0,"Chalet unifamiliar en venta en Soneiro, 1","Soneiro, 1. Alborelle (Sada)",43.327100,-8.287360,1.300 m²,3,NaN,1.300 m²,490 m²,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# NP.NAN

In [89]:
fig = px.bar([df_[x].dropna().count() for x in df_.columns], y = df_.columns, x = [df_[x].dropna().count() for x in df_.columns])

fig.update_layout(height = 1000)

In [90]:
def nanPercentage(col):
    orig_size = len(col)
    drop_size = len(col.dropna())
    return 1 - drop_size/orig_size

def nanReport(threshold):
    df = df_[[col for col in df_.columns if nanPercentage(df_[col]) < threshold]]
    print(f'Porcentaje de valores perdidos en total: {1- df.dropna().shape[0] / df.shape[0]}')
    print(f'Columnas conservadas: {len(df.columns)}')

# CONCLUSION: HAY MUCHOS NANS

In [91]:
nanReport(0.1)

Porcentaje de valores perdidos en total: 0.06826262147428297
Columnas conservadas: 9
